# (Opsiyonal) Olist reviews' - Translations...

* 🇧🇷 Brezilya Portekizcesi bilmiyorsanız, hadi `order_reviews` metinlerini 🇬🇧 İngilizce’ye çevirelim!

* Bunun için `pip install googletrans==4.0.2` yüklemeniz gerekecek.

☢️ Bu API ile herhangi bir sorun yaşarsanız, bunu düzeltmek için zaman harcamayın, bu package oldukça dengesiz… Aklınızda bulunsun:
- bu optional bir challenge
- Brezilya Portekizcesi ile yazılmış review’ların anlamını görmek için bazı yorumları Google Translate’e kopyalayıp yapıştırarak yine de eski yöntemle yapabilirsiniz.

In [16]:
pip install -U google-trans-new

Note: you may need to restart the kernel to use updated packages.


## Review Translator

👉 `reviews` dataset’ini load edin ve 1-yıldız rating’e sahip review’lardan bir örnek (sample) seçin.

In [17]:
# Verileri yükle
from olist.data import Olist
data = Olist().get_data()

👀 20 adet düşük puan alan yorumdan oluşan bir örneklem seçin (rastgele) ve bunu bir listeye dönüştürün.

In [18]:
from olist.data import Olist

ONE_STAR = 1
TEXT_COL = "review_comment_message"
RATING_COL = "review_score"

data = Olist().get_data()

# order_reviews dataframe'i al
reviews = data["order_reviews"].copy()

# 1 yıldız + dolu metin
one_star = reviews.loc[
    (reviews[RATING_COL] == ONE_STAR) &
    (reviews[TEXT_COL].notna()) &
    (reviews[TEXT_COL].astype(str).str.strip() != "")
].copy()

# 20 rastgele örnek
sample_20 = one_star.sample(n=20, random_state=42)

# listeye çevir
sample_list = sample_20[TEXT_COL].astype(str).tolist()

sample_list

['Demora de entrega',
 'Depois que fiz a compra, passou 30 dias para a loja me informar que não tem mais a mercadoria ',
 'ainda não entregaram meu produto, apenas o outros que avia comprado junto, mas o cabo HDMI NÃO foi entregue',
 'Não estou satisfeita, pois ainda não recebi o produto.',
 'Até hoje não recebi meu produto. Vou fazer uma reclamação no Procon e procurar meus direitos.',
 'O produto já passou da data de entrega e não consigo saber o que aconteceu. Até agora nada. ',
 'Até o momento não recebi o produto , não entraram em contato para falar o q houve',
 'Produto veio faltando duas peças e eu pedi a troca e até agora nada nem resposta',
 'comprei dois produtos há mais de um mês e não recebi nenhum',
 'Solicitei informações acerca da entrega de meu produto e fui informado que o recebeu dentro do prazo avençado, contudo hoje (19/12/2017) ainda não o recebi.',
 'Comprei dois produtos em um só número de pedido, só recebi o forro do sofá mas a cortina não chegou. Só que consta 

🗣 Bu göreve başlamadan önce önceden yüklediğiniz `google_translator` paketini kullanarak bu yorumları çevirin:

In [19]:
from googletrans import Translator

async def translate_list(sample_list):
    translator = Translator()
    out = []
    for t in sample_list:
        res = await translator.translate(t, src="pt", dest="en")
        out.append(res.text)
    return out

translated_en = await translate_list(sample_list)
translated_en[:20]

['Delivery delay',
 'After I made the purchase, it took 30 days for the store to inform me that they no longer had the merchandise.',
 "They still haven't delivered my product, just the others I purchased together, but the HDMI cable was NOT delivered",
 "I'm not satisfied as I haven't received the product yet.",
 "To this day I have not received my product. I'm going to file a complaint with Procon and look for my rights.",
 "The product is past its delivery date and I can't figure out what happened. So far nothing.",
 'So far I have not received the product, they have not contacted me to say what happened',
 'Product arrived missing two pieces and I asked for an exchange and so far nothing or response',
 "I bought two products over a month ago and I haven't received any of them.",
 "I requested information about the delivery of my product and was informed that I received it within the agreed deadline, however today (12/19/2017) I still haven't received it.",
 "I bought two products u

**Insights** 💡
- Bazı kötü review’lar delivery ile ilgili (`wait_time`, kaçırılan teslim tarihi, vb.)
- Ancak bazı kötü review’lar seller veya ürünle ilgili...

Peki iki olası nedeni nasıl ayırt edebiliriz?

Bu Olist’in mutlaka bilmesi gereken bir şey:
- bazı ürünleri katalogdan mı çıkarmalı?
- yoksa bazı seller’ları marketplace’ten mi kaldırmalı?


* 20 adet 1-yıldızlı review’un İngilizce çevirilerine bakıldığında şikayetlerin iki ana eksende toplandığı görülmektedir: (i) lojistik/teslimat kaynaklı sorunlar ve (ii) seller/ürün kaynaklı sorunlar. Örneklemde baskın tema teslimat gecikmesi veya hiç teslim edilmeme (örn. “haven’t received”, “past delivery date”, “no update”, “post office delay”). Bu tip yorumlar genellikle yüksek wait_time ve pozitif delay_vs_expected ile uyumludur ve farklı seller/ürünlere yaygın şekilde görülür. Bu durumda problem lojistik süreç (carrier, last-mile, takip/iletişim) kaynaklıdır; çözüm, SLA iyileştirme, taşıyıcı performans yönetimi ve müşteri bilgilendirmesinin güçlendirilmesi olmalıdır.
Buna ek olarak daha az sayıda ama belirgin seller/ürün kaynaklı şikayetler de vardır: yanlış ürün gönderimi, eksik parça, defolu ürün ve iade/değişim sürecinde iletişim sorunları (örn. “wrong product”, “missing pieces”, “defective”, “no response”). Bu tip yorumlar çoğunlukla lojistik metrikler normal olsa bile ortaya çıkar ve genellikle belirli seller’lar veya belirli SKU’lar üzerinde kümelenir. Bu durumda Olist için doğru aksiyon seller governance (uyarı/ceza/askıya alma), fulfillment doğruluğu kontrolleri ve ürün/kategori kalite denetimidir; tekrarlayan hatalarda problemli ürünlerin katalogdan çıkarılması da gündeme gelmelidir.